In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb 

loaded_model = xgb.XGBClassifier()
loaded_model.load_model('xgb_model_good.json')
cleaned_df = pd.read_csv('ufc_cleaned.csv')
fighters_df = pd.read_csv('ufc_fighters_cleaned.csv')
features_df = pd.read_csv('ufc_features.csv')

In [4]:
def getData(p1, p2, eventDate, ref): # Takes Two fighters, event date, and referee, and creates the features again for the model
    cols = ['height','weight','reach','SLpM','Str. Acc.', 'SApM','Str. Def','TD Avg.','TD Acc.','TD Def.','Sub. Avg.']
    
    row1 = fighters_df[fighters_df['name'] == p1]
    row2 = fighters_df[fighters_df['name'] == p2]

    if row1.empty or row2.empty:
        missing = p1 if row1.empty else p2
        raise ValueError(f"Fighter not found: {missing}")

    f1 = row1[cols].squeeze().to_numpy(dtype=float)
    f2 = row2[cols].squeeze().to_numpy(dtype=float)
    fighter_diff = f1 - f2


    #calculate age at event
    eventDate = pd.to_datetime(eventDate)
    p1Dob = pd.to_datetime(fighters_df.loc[fighters_df['name'] == p1, 'dob'].iloc[0])
    p2Dob = pd.to_datetime(fighters_df.loc[fighters_df['name'] == p2, 'dob'].iloc[0])
    p1Age = (eventDate - p1Dob).days / 365.25
    p2Age = (eventDate - p2Dob).days / 365.25
    ageDiff = p1Age - p2Age


    #calculate age adjusted stats
    age_adjust_cols = ['SLpM','Str. Acc.', 'SApM','Str. Def','TD Avg.','TD Acc.','TD Def.','Sub. Avg.']
    p1_age_adjusted = {}
    p2_age_adjusted = {}
    
    for col in age_adjust_cols:
        if col in row1.columns and col in row2.columns:
            p1_value = row1[col].squeeze()
            p2_value = row2[col].squeeze()
            
            # Apply age adjustment formula
            p1_age_adjusted[col] = p1_value * (1/p1Age)
            p2_age_adjusted[col] = p2_value * (1/p2Age)



    #find days since last fight
    p1_fights = cleaned_df[(cleaned_df['p1_fighter'] == p1) | (cleaned_df['p2_fighter'] == p1)]
    p2_fights = cleaned_df[(cleaned_df['p1_fighter'] == p2) | (cleaned_df['p2_fighter'] == p2)]
    
    p1_last = pd.to_datetime(p1_fights['event_date']).max() if not p1_fights.empty else pd.NaT
    p2_last = pd.to_datetime(p2_fights['event_date']).max() if not p2_fights.empty else pd.NaT
    
    p1_days = (eventDate - p1_last).days if pd.notnull(p1_last) else None
    p2_days = (eventDate - p2_last).days if pd.notnull(p2_last) else None
    days_diff = p1_days - p2_days


    #get stance and encode it
    stance1 = row1['stance'].squeeze()
    stance2 = row2['stance'].squeeze()
    categories = ['Open Stance', 'Orthodox', 'Sideways', 'Southpaw', 'Switch']
    stance1 = [True if stance1 == cat else False for cat in categories]
    stance2 = [True if stance2 == cat else False for cat in categories]


    #calculate records
    p1_win_fights = cleaned_df[((cleaned_df['p1_fighter'] == p1) & (cleaned_df['winner'] == 1)) | ((cleaned_df['p2_fighter'] == p1) & (cleaned_df['winner'] == 0))]
    p1_loss_fights = cleaned_df[((cleaned_df['p1_fighter'] == p1) & (cleaned_df['winner'] == 0)) | ((cleaned_df['p2_fighter'] == p1) & (cleaned_df['winner'] == 1))]

    p1_wins = len(p1_win_fights)
    p1_losses = len(p1_loss_fights)
    p1_total = p1_wins + p1_losses
    
    p2_win_fights = cleaned_df[((cleaned_df['p1_fighter'] == p2) & (cleaned_df['winner'] == 0)) | ((cleaned_df['p2_fighter'] == p2) & (cleaned_df['winner'] == 1))]
    p2_loss_fights = cleaned_df[((cleaned_df['p1_fighter'] == p2) & (cleaned_df['winner'] == 1)) | ((cleaned_df['p2_fighter'] == p2) & (cleaned_df['winner'] == 0))]

    p2_wins = len(p2_win_fights)
    p2_losses = len(p2_loss_fights)
    p2_total = p2_wins + p2_losses
    
    # Calculate differences
    win_diff = p1_wins - p2_wins
    loss_diff = p1_losses - p2_losses
    total_diff = p1_total - p2_total


    # calculate win streaks
    p1_past_fights = cleaned_df[
        (cleaned_df['event_date'].notna()) & 
        ((cleaned_df['p1_fighter'] == p1) | (cleaned_df['p2_fighter'] == p1))
    ].sort_values('event_date', ascending=False)
    
    p2_past_fights = cleaned_df[
        (cleaned_df['event_date'].notna()) & 
        ((cleaned_df['p1_fighter'] == p2) | (cleaned_df['p2_fighter'] == p2))
    ].sort_values('event_date', ascending=False)
    
    # Calculate p1's current win streak
    p1_win_streak = 0
    for _, fight in p1_past_fights.iterrows():
        # Check if p1 won this fight
        if (fight['p1_fighter'] == p1 and fight['winner'] == 1) or (fight['p2_fighter'] == p1 and fight['winner'] == 0):
            p1_win_streak += 1
        else:
            # Stop counting at first loss
            break
    
    # Calculate p2's current win streak
    p2_win_streak = 0
    for _, fight in p2_past_fights.iterrows():
        # Check if p2 won this fight
        if (fight['p1_fighter'] == p2 and fight['winner'] == 1) or (fight['p2_fighter'] == p2 and fight['winner'] == 0):
            p2_win_streak += 1
        else:
            # Stop counting at first loss
            break
    

    #referee
    ref_counts = (cleaned_df['referee'].value_counts()).get(ref, 0)


    #calculate EMA
    features = [
    'KD', 'SIG_STR_PCT', 'TD_PCT', 'SUB_ATT', 'REV', 'CTRL',
    'R1_KD', 'R1_SIG_STR_PCT', 'R1_TD_PCT', 'R1_SUB_ATT', 'R1_REV', 'R1_CTRL',
    'SIG_STR_PCT_DETAILED', 'R1_SIG_STR_PCT_DETAILED',
    'SIG_STR_LANDED', 'SIG_STR_ATTEMPTED', 'TOTAL_STR_LANDED', 'TOTAL_STR_ATTEMPTED',
    'TD_LANDED', 'TD_ATTEMPTED',
    'R1_SIG_STR_LANDED', 'R1_SIG_STR_ATTEMPTED', 'R1_TOTAL_STR_LANDED', 'R1_TOTAL_STR_ATTEMPTED',
    'R1_TD_LANDED', 'R1_TD_ATTEMPTED',
    'HEAD_LANDED', 'HEAD_ATTEMPTED', 'BODY_LANDED', 'BODY_ATTEMPTED',
    'LEG_LANDED', 'LEG_ATTEMPTED',
    'DISTANCE_LANDED', 'DISTANCE_ATTEMPTED', 'CLINCH_LANDED', 'CLINCH_ATTEMPTED',
    'GROUND_LANDED', 'GROUND_ATTEMPTED',
    'R1_HEAD_LANDED', 'R1_HEAD_ATTEMPTED', 'R1_BODY_LANDED', 'R1_BODY_ATTEMPTED',
    'R1_LEG_LANDED', 'R1_LEG_ATTEMPTED',
    'R1_DISTANCE_LANDED', 'R1_DISTANCE_ATTEMPTED', 'R1_CLINCH_LANDED', 'R1_CLINCH_ATTEMPTED',
    'R1_GROUND_LANDED', 'R1_GROUND_ATTEMPTED'
    ]   
    
    # Initialize dictionaries to store EMAs
    p1_emas = {}
    p2_emas = {}
    
    # Calculate EMAs for p1
    cleaned_df['event_date'] = pd.to_datetime(cleaned_df['event_date'])
    p1_prev_fights = cleaned_df[(cleaned_df['event_date'] < eventDate) & ((cleaned_df['p1_fighter'] == p1) | (cleaned_df['p2_fighter'] == p1))]
    p1_prev_fights = p1_prev_fights.sort_values('event_date', ascending=False).head(3)
    
    if not p1_prev_fights.empty:
        # Process each fight for p1
        for _, fight in p1_prev_fights.iterrows():
            position = 'p1' if fight['p1_fighter'] == p1 else 'p2'
            prefix = 'p1_' if position == 'p1' else 'p2_'
            
            # Extract feature values for this fight
            for feat in features:
                col_name = f"{prefix}{feat}"
                if col_name in fight and not pd.isna(fight[col_name]):
                    # Convert to numeric and add to list
                    if feat not in p1_emas:
                        p1_emas[feat] = []
                    
                    p1_emas[feat].append(pd.to_numeric(fight[col_name], errors='coerce'))

        for feat in features:
            if feat in p1_emas:
                if len(p1_emas[feat]) == 1:
                    # Only one fight, use that value
                    p1_emas[feat] = p1_emas[feat][0]
                elif len(p1_emas[feat]) == 2:
                    # Two fights, weight more recent fight higher (0.6, 0.4)
                    p1_emas[feat] = 0.6 * p1_emas[feat][0] + 0.4 * p1_emas[feat][1]
                elif len(p1_emas[feat]) >= 3:
                    # Three or more fights, use weights 0.5, 0.3, 0.2 for most recent to oldest
                    p1_emas[feat] = 0.5 * p1_emas[feat][0] + 0.3 * p1_emas[feat][1] + 0.2 * p1_emas[feat][2]
            else:
                p1_emas[feat] = np.nan
    
    # Calculate EMAs for p2
    p2_prev_fights = cleaned_df[(cleaned_df['event_date'] < eventDate) & 
                               ((cleaned_df['p1_fighter'] == p2) | (cleaned_df['p2_fighter'] == p2))]
    p2_prev_fights = p2_prev_fights.sort_values('event_date', ascending=False).head(3)
    
    if not p2_prev_fights.empty:
        # Process each fight for p2
        for _, fight in p2_prev_fights.iterrows():
            position = 'p1' if fight['p1_fighter'] == p2 else 'p2'
            prefix = 'p1_' if position == 'p1' else 'p2_'
            
            # Extract feature values for this fight
            for feat in features:
                col_name = f"{prefix}{feat}"
                if col_name in fight and not pd.isna(fight[col_name]):
                    # Convert to numeric and add to list
                    if feat not in p2_emas:
                        p2_emas[feat] = []
                    
                    p2_emas[feat].append(pd.to_numeric(fight[col_name], errors='coerce'))
        
        # Calculate simple weighted average for the last 3 fights (most recent has highest weight)
        for feat in features:
            if feat in p2_emas:
                if len(p2_emas[feat]) == 1:
                    # Only one fight, use that value
                    p2_emas[feat] = p2_emas[feat][0]
                elif len(p2_emas[feat]) == 2:
                    # Two fights, weight more recent fight higher (0.6, 0.4)
                    p2_emas[feat] = 0.6 * p2_emas[feat][0] + 0.4 * p2_emas[feat][1]
                elif len(p2_emas[feat]) >= 3:
                    # Three or more fights, use weights 0.5, 0.3, 0.2 for most recent to oldest
                    p2_emas[feat] = 0.5 * p2_emas[feat][0] + 0.3 * p2_emas[feat][1] + 0.2 * p2_emas[feat][2]
            else:
                p2_emas[feat] = np.nan
        
        feature_dict = {
        'winner': np.nan,  # Placeholder for prediction
        
        # Basic stats for fighter 1
        'p1_height': f1[0],
        'p1_weight': f1[1],
        'p1_reach': f1[2],
        'p1_slpm': f1[3],
        'p1_str_acc': f1[4],
        'p1_sapm': f1[5],
        'p1_str_def': f1[6],
        'p1_td_avg': f1[7],
        'p1_td_acc': f1[8],
        'p1_td_def': f1[9],
        'p1_sub_avg': f1[10],
        
        # Basic stats for fighter 2
        'p2_height': f2[0],
        'p2_weight': f2[1],
        'p2_reach': f2[2],
        'p2_slpm': f2[3],
        'p2_str_acc': f2[4],
        'p2_sapm': f2[5],
        'p2_str_def': f2[6],
        'p2_td_avg': f2[7],
        'p2_td_acc': f2[8],
        'p2_td_def': f2[9],
        'p2_sub_avg': f2[10],
        
        # Age and physical differences
        'p1_age_at_event': p1Age,
        'p2_age_at_event': p2Age,
        'height_diff': f1[0] - f2[0],
        'reach_diff': f1[1] - f2[1],
        'weight_diff': f1[2] - f2[2],
        'age_diff': ageDiff,
        
        # Skill differences
        'slpm_diff': f1[3] - f2[3],
        'stracc_diff': f1[4] - f2[4],
        'sapm_diff': f1[5] - f2[5],
        'strdef_diff': f1[6] - f2[6],
        'tdavg_diff': f1[7] - f2[7],
        'tdacc_diff': f1[8] - f2[8],
        'tddef_diff': f1[9] - f2[9],
        'subavg_diff': f1[10] - f2[10],
        
        # Time since last fight
        'p1_days_since_last_fight': p1_days,
        'p2_days_since_last_fight': p2_days,
        'days_since_last_fight_diff': days_diff,
        
        # Fight records
        'p1_wins': p1_wins,
        'p1_losses': p1_losses,
        'p1_total': p1_total,
        'p2_wins': p2_wins,
        'p2_losses': p2_losses,
        'p2_total': p2_total,
        'win_diff': win_diff,
        'loss_diff': loss_diff,
        'total_diff': total_diff,
        'p1_win_streak': p1_win_streak,
        'p2_win_streak': p2_win_streak,
        
        # Age adjusted stats
        'p1_age_adjusted_slpm': p1_age_adjusted['SLpM'],
        'p2_age_adjusted_slpm': p2_age_adjusted['SLpM'],
        'p1_age_adjusted_str_acc': p1_age_adjusted['Str. Acc.'],
        'p2_age_adjusted_str_acc': p2_age_adjusted['Str. Acc.'],
        'p1_age_adjusted_sapm': p1_age_adjusted['SApM'],
        'p2_age_adjusted_sapm': p2_age_adjusted['SApM'],
        'p1_age_adjusted_str_def': p1_age_adjusted['Str. Def'],
        'p2_age_adjusted_str_def': p2_age_adjusted['Str. Def'],
        'p1_age_adjusted_td_avg': p1_age_adjusted['TD Avg.'],
        'p2_age_adjusted_td_avg': p2_age_adjusted['TD Avg.'],
        'p1_age_adjusted_td_acc': p1_age_adjusted['TD Acc.'],
        'p2_age_adjusted_td_acc': p2_age_adjusted['TD Acc.'],
        'p1_age_adjusted_td_def': p1_age_adjusted['TD Def.'],
        'p2_age_adjusted_td_def': p2_age_adjusted['TD Def.'],
        'p1_age_adjusted_sub_avg': p1_age_adjusted['Sub. Avg.'],
        'p2_age_adjusted_sub_avg': p2_age_adjusted['Sub. Avg.']
    }
    
    # Add all EMA metrics for both fighters
    for feat in features:
        feature_dict[f'p1_{feat.lower()}_ema'] = p1_emas[feat]
        feature_dict[f'p2_{feat.lower()}_ema'] = p2_emas[feat]
    
    # Add stance encoding
    for i, stance_cat in enumerate(categories):
        feature_dict[f'p1_stance_{stance_cat}'] = stance1[i]
    for i, stance_cat in enumerate(categories):
        feature_dict[f'p2_stance_{stance_cat}'] = stance2[i]
    
    # Add referee frequency
    feature_dict['referee_freq'] = ref_counts

    # Convert to DataFrame
    prediction_df = pd.DataFrame([feature_dict])
    
    return prediction_df

def convert_to_american_odds(probability):
    if probability <= 0 or probability >= 1:
        raise ValueError("Probability must be between 0 and 1 exclusively")
    
    # For favorites (probability > 0.5)
    if probability > 0.5:
        odds = round(-100 * (probability / (1 - probability)))
        return f"{odds}"
    # For underdogs (probability <= 0.5)
    else:
        odds = round(100 * ((1 - probability) / probability))
        return f"+{odds}"

def predict_winner(p1, p2, eventDate, ref): # uses the features we calculated and inputs it into a model to get the outcome
    # Get feature data
    fight_features = getData(p1, p2, eventDate, ref)
    
    # Make prediction
    # Convert to float to ensure proper handling of numeric values
    fight_features_numeric = fight_features.drop(columns=['winner']).astype(float)
    prediction = loaded_model.predict_proba(fight_features_numeric)
    
    # Format results
    result = {
        'p1_name': p1,
        'p2_name': p2,
        'p1_win_probability': float(prediction[0][1]), # Probability fighter 1 wins
        'p2_win_probability': float(prediction[0][0]), # Probability fighter 2 wins
        'p1_american_odds': convert_to_american_odds(float(prediction[0][1])),
        'p2_american_odds': convert_to_american_odds(float(prediction[0][0])),
        'predicted_winner': p1 if prediction[0][1] > prediction[0][0] else p2
    }
    return result

prediction = predict_winner('Julian Erosa', 'Melquizael Costa', '2025-05-17', 'Jason Herzog')
print(f"Predicted winner: {prediction['predicted_winner']}")
print(f"{prediction['p1_name']} win probability: {prediction['p1_win_probability']:.2%}")
print(f"{prediction['p2_name']} win probability: {prediction['p2_win_probability']:.2%}")
print(f"{prediction['p1_name']} betting odds: {prediction['p1_american_odds']}")
print(f"{prediction['p2_name']} betting odds: {prediction['p2_american_odds']}")

Predicted winner: Julian Erosa
Julian Erosa win probability: 57.53%
Melquizael Costa win probability: 42.47%
Julian Erosa betting odds: -135
Melquizael Costa betting odds: +135


In [ ]:
gilbert_burns_rows = features_df[features_df['p1_fighter'] == 'Gilbert Burns']
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
gilbert_burns_rows.head(20)


,p1_fighter,p2_fighter,referee,winner,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,...,p1_R1_DISTANCE_ATTEMPTED_EMA,p2_R1_DISTANCE_ATTEMPTED_EMA,p1_R1_CLINCH_LANDED_EMA,p2_R1_CLINCH_LANDED_EMA,p1_R1_CLINCH_ATTEMPTED_EMA,p2_R1_CLINCH_ATTEMPTED_EMA,p1_R1_GROUND_LANDED_EMA,p2_R1_GROUND_LANDED_EMA,p1_R1_GROUND_ATTEMPTED_EMA,p2_R1_GROUND_ATTEMPTED_EMA
338,Gilbert Burns,Sean Brady,Herb Dean,0,70.0,170.0,71.0,Orthodox,3.17,0.48,...,28.828043,26.188976,1.551897,0.519685,2.582098,0.527559,1.000786,0.629921,1.258493,0.637795
616,Gilbert Burns,Jack Della Maddalena,Dan Miragliotta,0,70.0,170.0,71.0,Orthodox,3.17,0.48,...,28.656087,44.444444,0.103794,4.285714,0.164195,5.301587,1.001571,1.317460,1.516987,2.079365
1104,Gilbert Burns,Jorge Masvidal,Marc Goddard,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,16.624335,22.729409,0.415177,1.871028,0.656780,2.924977,3.006288,2.387605,3.067948,3.429231
1216,Gilbert Burns,Neil Magny,Fernando Portella,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,29.248694,26.219180,0.830354,2.734628,1.313561,3.741481,4.012577,1.549390,4.135899,2.053964
1617,Gilbert Burns,Khamzat Chimaev,Mike Beltran,0,70.0,170.0,71.0,Orthodox,3.17,0.48,...,20.497354,1.533333,0.660708,0.000000,1.627123,0.000000,5.025158,21.266667,5.271802,24.933333
1993,Gilbert Burns,Stephen Thompson,Marc Goddard,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,33.994812,52.993118,1.321421,0.526955,3.254259,0.535683,5.050316,0.012573,5.543606,0.015869
2927,Gilbert Burns,Aleksei Kunchenko,Keith Peterson,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,44.836630,33.666667,3.285958,0.666667,4.136508,0.666667,4.610012,0.000000,5.395360,0.000000
3088,Gilbert Burns,Mike Davis,Keith Peterson,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,20.322581,NaN,1.141740,NaN,1.543500,NaN,9.444770,NaN,12.588465,NaN
3089,Gilbert Burns,Mike Davis,Keith Peterson,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,20.322581,NaN,1.141740,NaN,1.543500,NaN,9.444770,NaN,12.588465,NaN
3600,Gilbert Burns,Dan Moret,Nic Jones,1,70.0,170.0,71.0,Orthodox,3.17,0.48,...,31.669291,NaN,4.157480,NaN,6.385827,NaN,1.496063,NaN,2.629921,NaN


In [8]:
gilbert_burns_rows = features_df[features_df['p2_fighter'] == 'Gilbert Burns']
gilbert_burns_rows.head(20)

,p1_fighter,p2_fighter,referee,winner,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,...,p1_R1_DISTANCE_ATTEMPTED_EMA,p2_R1_DISTANCE_ATTEMPTED_EMA,p1_R1_CLINCH_LANDED_EMA,p2_R1_CLINCH_LANDED_EMA,p1_R1_CLINCH_ATTEMPTED_EMA,p2_R1_CLINCH_ATTEMPTED_EMA,p1_R1_GROUND_LANDED_EMA,p2_R1_GROUND_LANDED_EMA,p1_R1_GROUND_ATTEMPTED_EMA,p2_R1_GROUND_ATTEMPTED_EMA
1052,Belal Muhammad,Gilbert Burns,Keith Peterson,1,71.0,170.0,72.0,Orthodox,4.39,0.43,...,53.208971,26.312172,0.224985,0.207588,0.340113,0.328390,5.140679,2.003143,8.015709,3.033974
2208,Kamaru Usman,Gilbert Burns,Herb Dean,1,72.0,170.0,76.0,Switch,4.36,0.52,...,40.972650,15.989349,6.857143,1.642847,7.494505,2.508507,1.707448,10.100710,1.754823,11.087297
2584,Tyron Woodley,Gilbert Burns,Herb Dean,0,69.0,170.0,74.0,Orthodox,2.36,0.48,...,10.139055,16.978729,4.300085,1.285684,4.554999,2.016999,0.544134,5.201270,0.645220,6.174444
2630,Demian Maia,Gilbert Burns,Osiris Maia,0,73.0,170.0,72.0,Southpaw,1.80,0.43,...,24.535750,26.958066,5.047912,2.571446,6.563562,4.034121,0.280527,2.402368,0.288760,3.348715
2850,Gunnar Nelson,Gilbert Burns,Neil Hall,0,71.0,170.0,72.0,Switch,1.77,0.61,...,2.865446,30.916616,1.077411,2.142840,1.116484,3.068124,0.859585,4.805030,0.879121,6.697839
3277,Olivier Aubin-Mercier,Gilbert Burns,Yves Lavigne,0,69.0,155.0,70.0,Southpaw,2.48,0.51,...,16.942326,30.665362,5.952102,1.283757,6.276637,2.088063,0.975562,0.872798,1.479961,2.156556
3488,Dan Hooker,Gilbert Burns,Mark Smith,1,72.0,155.0,75.0,Switch,5.03,0.48,...,36.694716,33.341176,0.111546,2.572549,0.197652,4.184314,0.516634,1.749020,0.524462,4.321569
3866,Jason Saggo,Gilbert Burns,Keith Peterson,0,71.0,155.0,71.0,Orthodox,2.50,0.39,...,17.193548,28.285714,0.000000,2.285714,0.258065,3.730159,4.935484,0.984127,6.548387,1.238095
5355,Andreas Stahl,Gilbert Burns,Jason Herzog,0,71.0,170.0,NaN,Orthodox,2.83,0.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
